In [2]:
import os
import pandas as pd
import datetime



In [3]:
os.getcwd()

'C:\\Users\\User'

In [4]:
os.chdir('C:\\Users\\User\\OneDrive - Tuskegee University\\Desktop\\PhD Projects\\Culling')

In [5]:
import pandas as pd

In [6]:
dataset= pd.read_excel('cdr.xlsx')

In [7]:
dataset.columns

Index(['HerdCode', 'StateCode_x', 'State', 'AnimalIndex', 'LeftHerdSeq',
       'CalvingDate', 'DaysSinceCalving', 'LactNum_x', 'DIM',
       'ProjActual305DayMilk', 'Proj305DayMEMilk', 'HerdmateAvgMilk', 'CAR1',
       'BredDate', 'NextCalvingOrDueDate', 'Diagnosis', 'StatusDate',
       'StatusCode', 'Culling', 'LeftDate', 'CalvingYear', 'CalvingMonth',
       'LeftYear', 'LeftMonth', 'CalvingSeason', 'CullingSeason',
       'BreedingYear', 'BreedingMonth', 'BreedingSeason', 'NumberOfCowsCalved',
       'CowRelative305ME', 'CowRelative305MECAT',
       'avgeMEmilkperherdperyearperseason', 'HerdRelative305ME',
       'HerdRelative305MECAT', 'CAR1.1', 'car_1', 'culling_reasons',
       'Herd_size', 'conception_date', 'cow_days', 'cow_days.1',
       'cow_risk_new', 'CDR'],
      dtype='object')

In [16]:
dataset_new=dataset[['CalvingDate','LeftDate','NextCalvingOrDueDate','HerdCode', 'StateCode_x', 'State', 'AnimalIndex', 'LeftHerdSeq', 'CDR', 'LactNum_x']]

In [17]:
dataset_new['start_date'] = dataset['NextCalvingOrDueDate'].fillna(dataset['LeftDate'])
dataset_new['end_date']=dataset['CalvingDate']

C:\Users\User\AppData\Local\Temp\ipykernel_25452\2559695667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_new['start_date'] = dataset['NextCalvingOrDueDate'].fillna(dataset['LeftDate'])
C:\Users\User\AppData\Local\Temp\ipykernel_25452\2559695667.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_new['end_date']=dataset['CalvingDate']


In [20]:
list_df=[]
for herd, data_her in dataset_new.groupby('HerdCode'):
    for Ani_, data_ani in data_her.groupby('AnimalIndex'):
        for left_, data_left in data_ani.groupby('LeftHerdSeq'):
            data_left['days_per_year'] = data_left.apply(days_in_each_year, axis=1)

# Expand the dictionary into separate columns for each year
            df_years = data_left['days_per_year'].apply(pd.Series)
            #print(df_years.reset_index())

# Concatenate the original dataframe with the new year columns
            df_final = pd.concat([data_left, df_years], axis=1).drop(columns=['days_per_year'])
            list_df.append(df_final)
            

            
            
            
            

In [23]:
final_df= pd.concat(list_df, axis=0)

In [27]:
final_df.to_csv('CDR_year.csv')

In [12]:
dataset.columns

Index(['HerdCode', 'StateCode_x', 'State', 'AnimalIndex', 'LeftHerdSeq',
       'CalvingDate', 'DaysSinceCalving', 'LactNum_x', 'DIM',
       'ProjActual305DayMilk', 'Proj305DayMEMilk', 'HerdmateAvgMilk', 'CAR1',
       'BredDate', 'NextCalvingOrDueDate', 'Diagnosis', 'StatusDate',
       'StatusCode', 'Culling', 'LeftDate', 'CalvingYear', 'CalvingMonth',
       'LeftYear', 'LeftMonth', 'CalvingSeason', 'CullingSeason',
       'BreedingYear', 'BreedingMonth', 'BreedingSeason', 'NumberOfCowsCalved',
       'CowRelative305ME', 'CowRelative305MECAT',
       'avgeMEmilkperherdperyearperseason', 'HerdRelative305ME',
       'HerdRelative305MECAT', 'CAR1.1', 'car_1', 'culling_reasons',
       'Herd_size', 'conception_date', 'cow_days', 'cow_days.1',
       'cow_risk_new', 'CDR'],
      dtype='object')

In [11]:
import pandas as pd
from datetime import datetime, timedelta

# Function to calculate days in each year between two dates
def days_in_each_year(row):
    end_date = row['start_date']
    start_date = row['end_date']
    
    current_date = start_date
    days_per_year = {}

    # Loop over each year
    while current_date.year <= end_date.year:
        start_of_year = datetime(current_date.year, 1, 1)
        end_of_year = datetime(current_date.year + 1, 1, 1) - timedelta(days=1)

        # Adjust the start and end dates if they are within the same year
        if current_date.year == start_date.year:
            start_of_year = start_date
        if current_date.year == end_date.year:
            end_of_year = end_date

        # Calculate days in the year
        days_in_year = (end_of_year - start_of_year).days + 1
        days_per_year[current_date.year] = days_in_year

        # Move to next year
        current_date = datetime(current_date.year + 1, 1, 1)
    
    return days_per_year


